In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
import sys
from pathlib import *

sys.path.append('../..')
from src.VipLauncher import VipLauncher
from src.VipLoader import VipLoader

Handshake with VIP

In [36]:
VipLauncher.init(api_key="VIP_API_KEY"); # Paste your VIP API key here


----------------------------------
| You are communicating with VIP |
----------------------------------



In [37]:
VipLauncher.show_pipeline("EGI")


Available pipelines
-------------------
BraTSPipeline_EGItuto/1.8.1
BraTSPipeline_EGItuto/1.9.0
-------------------


In [38]:
pipelines = {
    "v181": "BraTSPipeline_EGItuto/1.8.1", 
    "v190": "BraTSPipeline_EGItuto/1.9.0"
}

In [39]:
VipLauncher.show_pipeline(list(pipelines.values())[0])

--------------------------------------------
name: BraTSPipeline_EGItuto | version: 1.8.1
--------------------------------------------
pipeline_id: BraTSPipeline_EGItuto/1.8.1
--------------------------------------------
input_settings:
 - 't1ceImage': [File] Input structural T1-weighted post-contrast image
 - 't1Image': [File] Input structural T1-weighted pre-contrast image
 - 'skullStrip': [String] Flag whether to skull strip or not. Defaults to 1. This uses DeepMedic: https://cbica.github.io/CaPTk/seg_DL.html
 - 'brainTumor': [String] Flag whether to segment brain tumors or not. Defaults to 1. This uses DeepMedic: https://cbica.github.io/CaPTk/seg_DL.html
 - 'patientID': [Optional][String] Patient ID to pre-pend to final output file names. If empty, final output is of the form ${modality}_to_SRI.nii.gz
 - 'appliOutputDir': [String] Application output directory for final output
 - 't2Image': [File] Input structural T2-weighted contrast image
 - 'flImage': [File] Input structural FLAI

In [40]:
input_dir = PurePosixPath("/vip/EGI tutorial (group)/inputs")
subjects = VipLoader._list_dir(input_dir)

input_settings = {
    "t1Image": [(subject / "T1.nii.gz") for subject in subjects],
    "t1ceImage": [(subject / "T1GD.nii.gz") for subject in subjects],
    "t2Image": [(subject / "T2.nii.gz") for subject in subjects],
    "flImage": [(subject / "T2-FLAIR.nii.gz") for subject in subjects],
    "appliOutputDir": [str(subject.name) for subject in subjects],
    "brainTumor": "1",
    "skullStrip": "1",
    "patientID": 'No_value_provided'
}
input_settings

{'t1Image': [PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00019/T1.nii.gz'),
  PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00239/T1.nii.gz')],
 't1ceImage': [PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00019/T1GD.nii.gz'),
  PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00239/T1GD.nii.gz')],
 't2Image': [PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00019/T2.nii.gz'),
  PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00239/T2.nii.gz')],
 'flImage': [PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00019/T2-FLAIR.nii.gz'),
  PurePosixPath('/vip/EGI tutorial (group)/inputs/UPENN-GBM-00239/T2-FLAIR.nii.gz')],
 'appliOutputDir': ['UPENN-GBM-00019', 'UPENN-GBM-00239'],
 'brainTumor': '1',
 'skullStrip': '1',
 'patientID': 'No_value_provided'}

In [41]:
from time import time
start = time()

session = {}
# Launch the application
for version in pipelines:
    session[version] = VipLauncher(
        session_name = "repro-BraTS_%s" % version,
        output_dir = input_dir.parent / "outputs/tutorial" / version,
        pipeline_id = pipelines[version],
        input_settings = input_settings
    ).launch_pipeline()
    
# Monitor Executions
for version in pipelines:
    session[version].monitor_workflows()

print("\nElapsed time:", time() - start)


=== SESSION 'repro-BraTS_v181' ===

Output directory: '/vip/EGI tutorial (group)/outputs/tutorial/v181'
Pipeline ID: 'BraTSPipeline_EGItuto/1.8.1' --> checked
Input Settings --> parsed


=== LAUNCH PIPELINE ===

Parameter checks

----------------
Pipeline identifier: OK
Output directory: OK
Input settings: OK
----------------

Launching 1 new execution(s) on VIP
-------------------------------------
Execution Name: repro-BraTS_v181
Started Workflows:
	workflow-VpA1yW, 
-------------------------------------
Done.

=== SESSION 'repro-BraTS_v190' ===

Output directory: '/vip/EGI tutorial (group)/outputs/tutorial/v190'
Pipeline ID: 'BraTSPipeline_EGItuto/1.9.0' --> checked
Input Settings --> parsed


=== LAUNCH PIPELINE ===

Parameter checks

----------------
Pipeline identifier: OK
Output directory: OK
Input settings: OK
----------------

Launching 1 new execution(s) on VIP
-------------------------------------
Execution Name: repro-BraTS_v190
Started Workflows:
	workflow-vwosu9, 
------

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))